# Interactive .bib visualiser
This notebook takes a bib file and generates an interactive html page with each of the contained papers in chronological order.

In [3]:
from pylatexenc.latex2text import LatexNodes2Text 
import pybtex.database


bib_data = pybtex.database.parse_file('references.bib')


# create intermediary data structure (dict) holding all the values in an accessible way (for sorting by year later on)
all_useful_parts = []
for item in bib_data.entries.items():
    key = item[0]
    entry = item[1]
    url = entry.fields['url']
    title = entry.fields['title']
    year = entry.fields['year']
    authors = [[person.first_names, person.last_names] for person in entry.persons['author']]
    # make sure to catch any special characters in author's names using latex_to_text()
    authors = LatexNodes2Text().latex_to_text(", ".join([" ".join(f) + ' ' + " ".join(l)  for f, l in authors]))
    #print(authors, ':', title)
    first_author = entry.persons['author'][0].last_names[0]
    
    useful_parts = {'title':title, 'authors':authors, 'url':url, 'year':int(year), 'first_author':first_author}
    all_useful_parts.append(useful_parts)
    
    
# now the html can be built (by year [descending])
# NOTE: this may be improved using the beautifulsoup (bs4) python module? worth investigating
connected_papers_url = 'https://www.connectedpapers.com/search?q='
google_scholar_url = 'https://scholar.google.com/scholar?q='
all_html = ''
for year in range(2020, 2006, -1):
        
    this_years_papers = [paper for paper in all_useful_parts if (paper['year'] == year) ]
    
    html_row = '<div class="row">'
    html_row +='<div class="col-2 year"><h1>' + str(year) + '</h1></div>\n'
    
    #html_row += '<span>\n'
    for paper in this_years_papers:
        html_entry = '<span class="paper">\n'
        
        # first author's last name
        html_entry += '<a class="btn btn-primary first"><p>' + paper['first_author'] + '</p></a>\n'
        
        # paper title
        html_entry += '<p class="title"><a href="' + paper['url'] + '">' + paper['title'] + '</a></p>\n'
        
        # all authors
        html_entry += "<p class='authors'>"
        for author in paper['authors'].split(','):
            html_entry += '<a href="https://www.google.com/search?q=' + author + '">' + author + "</a>\n"
        html_entry += "</p>\n"
        
        # connected papers search link
        html_entry += '<a class="btn btn-info" href="' + connected_papers_url + paper['title'] + '"><p>Show Connected Papers</p></a>'
        
        # google scholar search link
        html_entry += '<a class="btn btn-info" href="' + google_scholar_url + paper['title'] + '"><p>Search on Scholar</p></a>'
        
        
        html_entry += '</span>'        
        html_row += html_entry
    #html_row += '</span>\n'
    
    html_row += '</div>\n'
    
    all_html += html_row
    
    
# finally, attach the header and footer html files to the all_html string
with open('header.html', 'r') as file:
    all_html = file.read() + all_html
with open('footer.html', 'r') as file:
    all_html += file.read()
    
# and save the resulting file as html
with open('index.html', 'w') as file:
    file.write(all_html)

# Behavioural Measures Plot